In [1]:
''' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'''
''' data is from http://help.sentiment140.com/for-students .  The Sentiment140 is used for brand management, polling, and planning a purchase. Sentiment140 is used to discover the sentiment of a brand or product or even a topic on the social media platform Twitter. Rather than working on keywords-based approach, which leverages high precision for lower recall, Sentiment140 works with classifiers built from machine learning algorithms. The Sentiment140 uses classification results for individual tweets along with the traditional surface that aggregated metrics. '''


' data is from http://help.sentiment140.com/for-students .  The Sentiment140 is used for brand management, polling, and planning a purchase. Sentiment140 is used to discover the sentiment of a brand or product or even a topic on the social media platform Twitter. Rather than working on keywords-based approach, which leverages high precision for lower recall, Sentiment140 works with classifiers built from machine learning algorithms. The Sentiment140 uses classification results for individual tweets along with the traditional surface that aggregated metrics. '

In [48]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [49]:
categories = [0, 1]

In [50]:

df = pd.read_csv('data/sentiment140.csv',header = None)

# drop emtpy records
df = df.dropna()

df.drop([1,2,3,4], axis=1, inplace=True)
# df.head()


X = df[5] #data
X.to_numpy()
# print(X)

y= df[0].replace(4, 1) #turn the 4 into a 1 to show positive sentiment
y.to_numpy()
print(y)
                                 

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: 0, Length: 1600000, dtype: int64


In [51]:
print(len(X)) #11541
print(len(y)) #11541

1600000
1600000


In [52]:
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
new_stopwords = ['a']
new_stopwords_list = stop_words.union(new_stopwords)

count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words = new_stopwords_list,  analyzer='word', max_df=1.0, min_df=1,  max_features=None) 
                             

count_vect.fit(X)

target_names = y

In [53]:
print(count_vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [54]:

X_counts = count_vect.transform(X)
print("The type of X_counts is {0}.".format(type(X_counts)))
print("The X matrix has {0} rows (documents) and {1} columns (words).".format(
        X_counts.shape[0], X_counts.shape[1]))

The type of X_counts is <class 'scipy.sparse.csr.csr_matrix'>.
The X matrix has 1600000 rows (documents) and 686637 columns (words).


In [55]:
tfidf_transformer = TfidfTransformer(use_idf=True)
tfidf_transformer.fit(X_counts)
X_tfidf = tfidf_transformer.transform(X_counts)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y.values, test_size=0.2, random_state=42)

In [57]:
print(X_train.shape)
print(y_train.shape)

(1280000, 686637)
(1280000,)


In [58]:
print(X_test.shape)
print(y_test.shape)

(320000, 686637)
(320000,)


In [59]:
nb_model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train, y_train);


In [60]:
# feature_words = count_vect.get_feature_names()
# n = 20 #number of top words associated with the category that we wish to see
# target_names[categories] = 1
# for cat in range(len(categories)): #categories == ['negative', 'positive']
#     # print(f"\nTarget: {cat}")
#     # print(f"\nname: ", target_names[cat])
#     print(f"\nTarget: {cat}, name:", target_names[cat])
#     log_prob = nb_model.feature_log_prob_[cat]
#     i_topn = np.argsort(log_prob)[::-1][:n]
#     features_topn = [feature_words[i] for i in i_topn]
#     print(f"Top {n} tokens: ", features_topn)


In [61]:
y_predicted = nb_model.predict(X_test)
# X_test[0]

In [62]:
# y_predicted

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predicted)


0.764128125

In [64]:
y_pred_proba = nb_model.predict_proba(X_test)[:,1]
y_pred_proba

array([0.72625202, 0.35755314, 0.5112429 , ..., 0.66616774, 0.27820323,
       0.29880966])

In [65]:
thresh=0.7
y_pred = (y_pred_proba>=thresh).astype(int)
# y_pred

In [66]:
y_pred

array([1, 0, 0, ..., 0, 0, 0])

0.7783251231527093

In [67]:
# pickle and save the model 
import pickle
save_documents = open("pickled_algos/pickled_nb_sentiment140.pickle","wb")
pickle.dump(nb_model, save_documents)
save_documents.close()

tfidf = tfidf_transformer
save_documents = open("pickled_algos/tfidf_transformer_sentiment140.pickle", "wb")
pickle.dump(tfidf, save_documents)
save_documents.close()

count_vect = count_vect
save_documents = open("pickled_algos/count_vect_sentiment140.pickle", "wb")
pickle.dump(count_vect, save_documents)
save_documents.close()



In [68]:
def predict_one_twitter(input):
    with open('pickled_algos/pickled_nb_sentiment140.pickle', 'rb') as f:
        model = pickle.load(f)

    with open('pickled_algos/tfidf_transformer_sentiment140.pickle', 'rb') as f:
        tfidf = pickle.load(f)

    with open('pickled_algos/count_vect_sentiment140.pickle', 'rb') as f:
        cv = pickle.load(f)

  
    cv_transformed = cv.transform(string_pred) #counts how many words
    tfidf_transformed = tfidf.transform(cv_transformed)  #tf == cv . 
    string_predicted = model.predict(tfidf_transformed) 
    res = str(string_predicted[0])
    if res == '0':
        res = ('Negative Sentiment')
    else:
        res = ("Positive Sentiment")
    return res

# string_pred = ['this girl is a foster pit and has none of her teeth']
# string_pred = ['fill out an online form to find this male puppy a forever home']
string_pred = ['this dog is great i love him']
predict_one_twitter(string_pred)

'Positive Sentiment'